In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

c:\Users\rafae\Documents\Projects\hands-on-llms-rafael\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct"
)

### Loading the model

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map='cuda',
    torch_dtype='auto',
    trust_remote_code=True,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.47s/it]


### Creating the pipeline

In [4]:
generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Testing our pipeline

In [ ]:
prompt="Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache=False)

print(output[0]['generated_text']) # type: ignore

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Mention the steps you're taking to prevent it in the future.

Email:

Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah,

I hope this email finds you well


### Checking the layers of the model

In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

#### Tokenizing the text

In [5]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids

#### Send tokens to GPU

In [8]:
input_ids = input_ids.to('cuda')

#### Forward pass

In [10]:
model_output = model.model(input_ids)

You are not running the flash-attention implementation, expect numerical differences.


#### Apply the LM Head to get logits about the vocabulary

In [12]:
lm_head_output = model.lm_head(model_output[0])

#### Most likely to be next token is...

In [14]:
token_id = lm_head_output[0,-1].argmax(-1)

In [15]:
tokenizer.decode(token_id)

'Paris'